# Homework 3

**Name:** Oscar Beltran Villegas

**e-mail:** oscar.beltran7944@alumnos.udg.mx

# MODULES

In [8]:
import math
import numpy as np

# CLASSES

In [6]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
       and also provides a bunch of high level functions
       """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y=None):
        if y is None:
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y

    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)

    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)

    # Rotate vector
    def rotated(self, angle):
        cos_theta = math.cos(angle)
        sin_theta = math.sin(angle)
        x = self.x * cos_theta - self.y * sin_theta
        y = self.x * sin_theta + self.y * cos_theta
        return Vec2d(x, y)

# FUNCTIONS

In [ ]:
def brownian_motion_3d(num_steps, speed, start_pos=(0, 0, 0)):
    """
    Function to simulate 3D Brownian motion
    Arguments:
        num_steps: Number of steps to simulate
        speed: Speed of the particle
        start_pos: Initial position of the particle
    Returns:
        trajectory: 3D trajectory of the particle
    """
    trajectory = np.zeros((num_steps + 1, 3)) # initialize the trajectory
    trajectory[0] = start_pos # set the initial position

    velocity = Vec2d(speed, 0) # initialize the velocity vector
    
    for i in range(1, num_steps + 1): # simulate the motion

        # generate a random angle to turn the velocity vector
        turn_angle = np.random.uniform(low=-np.pi, high=np.pi)
        velocity = velocity.rotated(turn_angle)
        
        # update the position
        trajectory[i] = trajectory[i-1] + np.array([velocity.x, velocity.y, velocity.get_length()])
    
    return trajectory